In [1]:
import cv2
import matplotlib.pyplot as plt

In [2]:
# Importing Models and set mean values
face1 = "/content/drive/MyDrive/Intel SIP/E_Auto/Copy of opencv_face_detector.pbtxt"
face2 = "/content/drive/MyDrive/Intel SIP/E_Auto/Copy of opencv_face_detector_uint8.pb"
age1 = "/content/drive/MyDrive/Intel SIP/E_Auto/Copy of age_deploy.prototxt"
age2 = "/content/drive/MyDrive/Intel SIP/E_Auto/Copy of age_net.caffemodel"
gen1 = "/content/drive/MyDrive/Intel SIP/E_Auto/Copy of gender_deploy.prototxt"
gen2 = "/content/drive/MyDrive/Intel SIP/E_Auto/Copy of gender_net.caffemodel"

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

# Face
face = cv2.dnn.readNet(face2, face1)

# age
age = cv2.dnn.readNet(age2, age1)

# gender
gen = cv2.dnn.readNet(gen2, gen1)
la = ['(25-32)', '(24-26)', '(18-22)', '(25-30)',
	'(25-32)', '(38-43)', '(48-53)', '(60-100)']
lg = ['Male', 'Female']


In [36]:
import os
for itop in os.listdir("/content/drive/MyDrive/Intel SIP/E_Auto/Faces"):
	image = cv2.imread("/content/drive/MyDrive/Intel SIP/E_Auto/Faces/"+itop,1)
	image = cv2.resize(image, (720, 640))
	fr_cv = image.copy()
	fr_h = fr_cv.shape[0]
	fr_w = fr_cv.shape[1]
	blob = cv2.dnn.blobFromImage(fr_cv, 1.0, (300, 300),
								[104, 117, 123], True, False)

	face.setInput(blob)
	detections = face.forward()
	faceBoxes = []
	for i in range(detections.shape[2]):

		confidence = detections[0, 0, i, 2]
		if confidence > 0.7:

			x1 = int(detections[0, 0, i, 3]*fr_w)
			y1 = int(detections[0, 0, i, 4]*fr_h)
			x2 = int(detections[0, 0, i, 5]*fr_w)
			y2 = int(detections[0, 0, i, 6]*fr_h)

			faceBoxes.append([x1, y1, x2, y2])

			cv2.rectangle(fr_cv, (x1, y1), (x2, y2),
						(0, 255, 0), int(round(fr_h/150)), 8)


	if not faceBoxes:
		print("No face detected")

	for faceBox in faceBoxes:

		face1 = fr_cv[max(0, faceBox[1]-15):
					min(faceBox[3]+15, fr_cv.shape[0]-1),
					max(0, faceBox[0]-15):min(faceBox[2]+15,
								fr_cv.shape[1]-1)]

		blob = cv2.dnn.blobFromImage(
			face1, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

		gen.setInput(blob)
		genderPreds = gen.forward()
		gender = lg[genderPreds[0].argmax()]
		age.setInput(blob)
		agePreds = age.forward()
		age1 = la[agePreds[0].argmax()]
		cv2.putText(fr_cv,
					f'{gender}, {age1}',
					(faceBox[0]-0, faceBox[1]-10),
					cv2.FONT_HERSHEY_SIMPLEX,
					1.3,
					(220, 0, 0),
					4,
					cv2.LINE_AA)
	plt.figure(figsize=(7, 7))
	plt.imshow(fr_cv)


Output hidden; open in https://colab.research.google.com to view.

#Overall accuracy around 85%.